# Toronto Neighborhoods Project
this project will consist in 3 sections.
   1. Web Scraping and dataframe formatting
   
       At section 1 the base dataframe is created, with the following columns:
        * Postal Code
        * Borough
        * Neighborhood
   
   2. Adding geographical coordinates
   
       Adding geographical coordinates, 2 options are approached. Using the geocoder package and merging the csv provided.
   
   3. Clustering and exploration of data
   
       The clustering and visualization of data is made by using the 4square API, and clustering data by the top venues on each neighborhood based on frequency
       
   4. Results
       
       Finally, an interpretation of the results is done



# Part 1: Web Scraping and dataframe formatting

In [1]:
#importing libraries:
import pandas as pd



#Config on visualization in display of dataframes:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
toronto = tables[0]

The next step is to clean the dataset. The following instructions were provided

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
# Dropping not assigned Boroughs
toronto.drop(toronto[toronto.Borough == 'Not assigned' ].index, inplace = True)
#Grouping neighborhoods with the same postal code
toronto.groupby(['Postal code','Borough'], sort = False).agg(lambda x: ','.join(x)).reset_index(level=['Postal code','Borough'])
# Replacing '/' with a comma
toronto['Neighborhood']=toronto['Neighborhood'].str.replace("/", ',' , regex = True)
# Replacing Neighborhood 'Not Assigned' values with the Boroughs, and reseting the index:
toronto.loc[toronto[toronto['Neighborhood'] == 'Not assigned'].index,'Neighborhood']= toronto[toronto['Neighborhood'] == 'Not assigned']['Borough']
toronto.reset_index(drop=True, inplace= True)

In [4]:
toronto.shape

(103, 3)

In [5]:
toronto.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


# Part 2: Adding geographical coordinates

In [6]:
#!conda install -c conda-forge geocoder

In [7]:
import geocoder # import geocoder

#Based on the code provided, a function to obtain the coordinates is created:
def getcoord(postal_code):
    # initialize your variable to None
    coords = None
    # loop until you get the coordinates
    while(coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      coords = g.latlng
    return coords[0],coords[1]

### Option 1: based on the function and the geocoder package

In [8]:
#toronto['Latitude'], tornonto['Longitude'] = zip(*toronto['Neighborhood'].apply(getcoord))

### Option 2: reading the cvs file provided by the instructor

In [9]:
coords = pd.read_csv('toronto_coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Sorting values in order to assign correctly the coordinates of the csv file. 
toronto.sort_values(by=['Postal code'], inplace=True)
toronto.reset_index(drop=True, inplace = True)
coords.sort_values(by=['Postal Code'], inplace=True)
coords.reset_index(drop=True, inplace= True)

toronto['Latitude'] = coords['Latitude']
toronto['Longitude'] = coords['Longitude']
toronto

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437


# Part 3: Clustering and exploration of data

In [11]:
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [12]:
# Visualizing neighborhoods in Toronto:
#Searched on google:
latitude = 43.651070
longitude= -79.347015
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Simplifying our dataset to East, West, Central and Downtown Toronto

In [13]:
toronto_simp = toronto[toronto['Borough'] == 'East Toronto'].append(toronto[toronto['Borough'] == 'West Toronto']).append(toronto[toronto['Borough'] == 'Central Toronto']).append(toronto[toronto['Borough'] == 'Downtown Toronto']).reset_index(drop=True) 

toronto_simp.head(15)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
5,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259
6,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750
7,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
8,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763
9,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325


### Visualizing The map, but with only toronto boroughs:

In [14]:
map_toronto_simp = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_simp['Latitude'], toronto_simp['Longitude'], toronto_simp['Borough'], toronto_simp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_simp)  
    
map_toronto_simp

## Clustering

In [15]:
#Setting up credential for foursquare
CLIENT_ID = 'VSUAGMIRY1XR25KQGAPWC1114QWZXBDTSJUWFAIJRNSPQLL4' # your Foursquare ID
CLIENT_SECRET = 'IUYB01IRZRYJ3VVHERTXNGPK11MQJJOERIMJBQVJMBGSMCCZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



In [44]:
#Function to obtain nearby venues of neighborhoods:
def getNearbyVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
#getting nearby venue of each Toronto neigh:
LIMIT = 100
toronto_venues = getNearbyVenues(names=toronto_simp['Neighborhood'],
                                   latitudes=toronto_simp['Latitude'],
                                   longitudes=toronto_simp['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Business reply mail Processing CentrE
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , The Junction South
Parkdale , Roncesvalles
Runnymede , Swansea
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Q

In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(1265, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
2,The Beaches,43.676357,-79.293031,Williamson Road Playground,43.674716,-79.297338,Playground
3,"The Danforth West , Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
4,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [47]:
#Counting venues for each neighborhood:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,18,18,18,18,18,18
"Brockton , Parkdale Village , Exhibition Place",16,16,16,16,16,16
Business reply mail Processing CentrE,15,15,15,15,15,15
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",12,12,12,12,12,12
Central Bay Street,79,79,79,79,79,79
Christie,11,11,11,11,11,11
Church and Wellesley,53,53,53,53,53,53
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,28,28,28,28,28,28


## Analyzing each neighborhood

In [48]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [49]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.055556,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.0000,0.055556,0.111111,0.055556,0.000000,0.000000,0.00,0.00,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.055556,0.055556,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0

In [50]:
toronto_grouped.shape

(37, 209)

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                     venue  freq
0             Cocktail Bar  0.11
1       Mexican Restaurant  0.06
2  Comfort Food Restaurant  0.06
3           Breakfast Spot  0.06
4                 Fountain  0.06


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Café  0.19
1      Breakfast Spot  0.12
2         Coffee Shop  0.12
3  Italian Restaurant  0.06
4                 Bar  0.06


----Business reply mail Processing CentrE----
           venue  freq
0    Yoga Studio  0.07
1  Auto Workshop  0.07
2     Smoke Shop  0.07
3            Spa  0.07
4  Burrito Place  0.07


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0      Airport Lounge  0.17
1    Airport Terminal  0.17
2             Airport  0.08
3  Airport Food Court  0.08
4         Coffee Shop  0.08


----Central Bay Street----
                venue  freq
0         Coffee Shop  

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Berczy Park,Cocktail Bar,Comfort Food Restaurant,Beer Bar,Grocery Store,French Restaurant,Italian Restaurant,Fountain
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Bar,Gym,Italian Restaurant
2,Business reply mail Processing CentrE,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Terminal,Airport,Coffee Shop,Airport Food Court,Airport Gate,Airport Service
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Café,Chinese Restaurant,Japanese Restaurant


## Clustering:


In [67]:
# set number of clusters
kclusters = 7

toronto_clusters = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clusters)

# check cluster labels generated for each row in the dataframe
print(toronto_grouped.shape)
kmeans.labels_


(37, 209)


array([0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 3, 1, 2,
       2, 0, 4, 6, 0, 0, 0, 0, 0, 2, 0, 5, 0, 0, 0])

In [68]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_simp

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged# check the last columns!
#toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged.dropna(inplace= True)
toronto_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,5.0,Other Great Outdoors,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Bookstore,Ice Cream Shop,Coffee Shop,Italian Restaurant,Café,Juice Bar
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0.0,Sandwich Place,Fast Food Restaurant,Board Shop,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Coffee Shop,Café,Gastropub,Bank,Fish Market,Diner,Italian Restaurant
4,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558,0.0,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place
5,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,0.0,Bakery,Gym / Fitness Center,Athletics & Sports,Middle Eastern Restaurant,Music Venue,Park,Pharmacy
6,M6J,West Toronto,"Little Portugal , Trinity",43.647927,-79.419750,0.0,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Men's Store,Asian Restaurant,Yoga Studio,Cocktail Bar
7,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,2.0,Café,Coffee Shop,Breakfast Spot,Yoga Studio,Bar,Gym,Italian Restaurant
8,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763,1.0,Park,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,M6S,West Toronto,"Runnymede , Swansea",43.651571,-79.484450,0.0,Café,Pizza Place,Coffee Shop,Sushi Restaurant,Pub,Juice Bar,Bar


# Part 4: Visualizing results

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='cartodbdark_matter')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finally, lets see the top venues of each cluster and analyze what differentiates them from the others

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,24,24,24,24,24,24,24,24
unique,4,14,15,19,18,19,18,22
top,Downtown Toronto,Coffee Shop,Café,Restaurant,Middle Eastern Restaurant,Italian Restaurant,Italian Restaurant,Restaurant
freq,14,8,6,3,3,4,4,2


## In cluster 0, you can find mainly coffe shops and restaurants, with a predominance of Italian food.

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,3,3,3,3,3,3,3,3
unique,2,1,1,1,1,1,1,1
top,Central Toronto,Park,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
freq,2,3,3,3,3,3,3,3


## In cluster 1, parks, yoga studios, and Event spaces are to be found. This cluster contains venues for more active people with taste on mediterranan and African food

In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,6,6,6,6,6,6,6,6
unique,3,2,6,5,5,6,6,6
top,Downtown Toronto,Coffee Shop,Breakfast Spot,Italian Restaurant,Sushi Restaurant,Distribution Center,Bank,Bank
freq,4,4,1,2,2,1,1,1


## In Cluster 2, Banks and sushi restaruant differentiates it from cluster 0

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1
top,Central Toronto,Park,Trail,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant
freq,1,1,1,1,1,1,1,1


## Cluster 3 is the best venue to practis trail. 

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1
top,Downtown Toronto,Bike Trail,Playground,Building,Campground,Yoga Studio,Diner,Event Space
freq,1,1,1,1,1,1,1,1


## Cluster 4, Bike trail venues, and playgrounds are most frequent. Better fit to have kids!

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1
top,East Toronto,Other Great Outdoors,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant
freq,1,1,1,1,1,1,1,1


## Cluster 5 is the ideal cluster for completely active people. 5 out of 7 top venues are for physical activities.

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].describe(include= 'object')

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
count,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1
top,Central Toronto,Health & Beauty Service,Gym,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
freq,1,1,1,1,1,1,1,1


## Cluster 6 is for people who take seriously their health and beaty rutines. 